In [1]:
import supervisely_lib as sly
import os

In [2]:
address = os.environ['SERVER_ADDRESS']
token = os.environ['API_TOKEN']

In [3]:
print("Server address: ", address)
print("Your API token: ", token)

Server address:  http://192.168.1.69:5555
Your API token:  OfaV5z24gEQ7ikv2DiVdYu1CXZhMavU7POtJw2iDtQtvGUux31DUyWTXW6mZ0wd3IRuXTNtMFS9pCggewQWRcqSTUi4EJXzly8kH7MJL1hm3uZeM2MCn5HaoEYwXejKT


In [4]:
api = sly.Api(address, token)

In [5]:
team = api.team.get_list()[0]
workspace = api.workspace.get_list(team.id)[0]
print("Current team: id = {}, name = {!r}".format(team.id, team.name))
print("Current workspace: id = {}, name = {!r}".format(workspace.id, workspace.name))

Current team: id = 9, name = 'max'
Current workspace: id = 9, name = 'my_super_workspace_002'


In [6]:
project_name = 'project_test'
if api.project.exists(workspace.id, project_name):
    project_name = api.project.get_free_name(workspace.id, project_name)
project = api.project.create(workspace.id, project_name, 'project is created using IPython and Supervisely API')
print('Project {!r} has been sucessfully created: '.format(project.name))
print(project)

Project 'project_test_006' has been sucessfully created: 
ProjectInfo(id=769, name='project_test_006', description='project is created using IPython and Supervisely API', size='0', readme='', workspace_id=9, created_at='2019-03-13T15:44:18.754Z', updated_at='2019-03-13T15:44:18.754Z')


In [9]:
images = sly.fs.list_files('/workdir/demo_data/images')
images

['/workdir/demo_data/images/cats.jpg',
 '/workdir/demo_data/images/gooses.jpg',
 '/workdir/demo_data/images/street.jpeg',
 '/workdir/demo_data/images/friends.jpeg']

In [10]:
# let's put first two images to the first dataset "ds1", and the rest of the images - to the second dataset "ds2
datasets = ['ds1', 'ds2']
dataset_images = [images[0:2], images[2:]]

for ds_name, img_paths in zip(datasets, dataset_images):
    ds = api.dataset.create(project.id, ds_name)
    print('Dataset {!r} has been sucessfully creates: id={}'.format(ds.name, ds.id))
    for img_path in img_paths:
        img_hash = api.image.upload_path(img_path)
        image_info = api.image.add(ds.id, sly.fs.get_file_name(img_path), img_hash)
        print('Image (id={}, name={}) has been sucessfully added'.format(image_info.id, image_info.name))

Dataset 'ds1' has been sucessfully creates: id=819
Image (id=81849, name=cats) has been sucessfully added
Image (id=81850, name=gooses) has been sucessfully added
Dataset 'ds2' has been sucessfully creates: id=820
Image (id=81851, name=street) has been sucessfully added
Image (id=81852, name=friends) has been sucessfully added


In [11]:
print("Number of images in created projects: ", api.project.get_images_count(project.id))

Number of images in created projects:  4


In [12]:
#define object classes
class_person = sly.ObjClass('person', sly.Rectangle, color=[255, 0, 0])
class_car = sly.ObjClass('car', sly.Polygon, color=[0, 255, 0])
class_road = sly.ObjClass('road', sly.Bitmap, color=[0, 0, 255])
obj_class_collection = sly.ObjClassCollection([class_person, class_car, class_road])

In [13]:
#define tags for images
tagmeta_weather = sly.TagMeta(name='weather',
                              value_type=sly.TagValueType.ONEOF_STRING,
                              possible_values=['rain', 'sun', 'cloud'],
                              color=[153, 0, 153])
tagmeta_annotate = sly.TagMeta('to_annotation', sly.TagValueType.NONE)
img_tag_meta_collection = sly.TagMetaCollection([tagmeta_weather, tagmeta_annotate])

In [14]:
#define tags for objects
tagmeta_vehicle_type = sly.TagMeta('vehicle_type', sly.TagValueType.ONEOF_STRING, ['sedan', 'suv', 'hatchback'])
tagmeta_confidence = sly.TagMeta('confidence', sly.TagValueType.ANY_NUMBER)
obj_tag_meta_collection = sly.TagMetaCollection([tagmeta_vehicle_type, tagmeta_confidence])

In [15]:
#combine everythiong to project meta
meta = sly.ProjectMeta(obj_class_collection, img_tag_meta_collection, obj_tag_meta_collection)
print(meta)
api.project.update_meta(project.id, meta.to_json())

ProjectMeta:

Object Classes:
Name:  person    Shape: Rectangle    Color: [255, 0, 0]
Name:  car       Shape: Polygon      Color: [0, 255, 0]
Name:  road      Shape: Bitmap       Color: [0, 0, 255]

Image Tags
Tag Metas:
Name:  weather                  Value type:oneof_string  Possible values:['rain', 'sun', 'cloud']
Name:  to_annotation            Value type:none          Possible values:None      

Object Tags
Tag Metas:
Name:  vehicle_type             Value type:oneof_string  Possible values:['sedan', 'suv', 'hatchback']
Name:  confidence               Value type:any_number    Possible values:None      


